In [1]:
import pandas as pd
import os
import requests
from datetime import datetime, timedelta
import io
import openpyxl

file_path = "C:\\Users\\dbcin\\OneDrive\\Desktop\\Consolidated Stock Data\\04-July-2024\\Consolidated Data for 04-07-2024.xlsx"
excel_data = pd.ExcelFile(file_path)

print(excel_data.sheet_names)

#Read each sheet into dataframe
data_frames = {}
for sheet in excel_data.sheet_names:
    data_frames[sheet] = pd.read_excel(file_path, sheet_name=sheet)

['Close', 'Open', 'High', 'Low', 'Volume']


In [2]:
def identify_stocks_with_empty_values(sheet_name, excel_data, num_rows=30):
    empty_stocks_info = set()
    df = pd.read_excel(excel_data, sheet_name=sheet_name)
    
    for col in df.columns[1:]:  # Skip the first column which is 'Date'
        if df[col].iloc[-num_rows:].isnull().any():
            empty_stocks_info.add(col)
    return empty_stocks_info
    
# Path to your Excel file
excel_path = file_path

# Identify stocks with empty values in 'Close', 'Open', 'High', and 'Low' sheets
empty_stocks_close = identify_stocks_with_empty_values('Close', excel_path)
empty_stocks_open = identify_stocks_with_empty_values('Open', excel_path)
empty_stocks_high = identify_stocks_with_empty_values('High', excel_path)
empty_stocks_low = identify_stocks_with_empty_values('Low', excel_path)
empty_stocks_volume = identify_stocks_with_empty_values('Volume', excel_path)

empty_stocks = empty_stocks_close.union(empty_stocks_open, empty_stocks_high, empty_stocks_low, empty_stocks_volume)
print(empty_stocks)

data_frames = {
    'Close': pd.read_excel(excel_path, sheet_name='Close'),
    'Open': pd.read_excel(excel_path, sheet_name='Open'),
    'High': pd.read_excel(excel_path, sheet_name='High'),
    'Low': pd.read_excel(excel_path, sheet_name='Low'),
    'Volume' : pd.read_excel(excel_path, sheet_name='Volume')
}

{'KRONOX', 'DEEDEV', 'IXIGO', 'AEGISLOG', 'AWFIS', 'BOROSCI', 'UNITDSPR', 'ABDL', 'GODIGIT', 'STANLEY'}


In [3]:
# Create separate dataframes for cleaned data
cleaned_data_frames = {
    'Close': data_frames['Close'].copy(),
    'Open': data_frames['Open'].copy(),
    'High': data_frames['High'].copy(),
    'Low': data_frames['Low'].copy(),
    'Volume': data_frames['Volume'].copy()
}

print("Head of Cleaned DataFrames:")
for sheet, df in cleaned_data_frames.items():
    print(f"\n{sheet} DataFrame:")
    print(df.reset_index().head())


def preprocess_dataframe(df):
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'])  # Convert 'Date' to datetime
        df.set_index('Date', inplace=True)      # Set 'Date' as index
        return df
    else:
        print(f"'Date' column not found in {df.name} dataframe.")
        return df

# Apply preprocessing to each dataframe in data_frames
for sheet, df in data_frames.items():
    data_frames[sheet] = preprocess_dataframe(df)

# Check if the 'Date' column is datetime in each dataframe
for sheet, df in data_frames.items():
    if 'Date' in df.index.names and pd.api.types.is_datetime64_any_dtype(df.index):
        print(f"'Date' column in {sheet} dataframe is of datetime data type and is indexed.")
    else:
        print(f"'Date' column in {sheet} dataframe is not properly set as datetime index.")

# Print head of cleaned dataframes
print("\nHead of Cleaned DataFrames:")
for sheet, df in data_frames.items():
    print(f"\n{sheet} DataFrame:")
    print(df.head())


#Define Check and Reindex

def check_and_reindex(df1, df2):
    if df1.index.equals(df2.index):
        print("The indices of the DataFrames are the same.")
    else:
        print("The indices of the DataFrames are not the same. Reindexing...")
        df2 = df2.reindex(df1.index)
        if df1.index.equals(df2.index):
            print("Reindexed and now the indices are the same.")
        else:
            print("Reindexing failed. The indices are still not the same.")
    return df2

Head of Cleaned DataFrames:

Close DataFrame:
   index        Date  20MICRONS      360ONE  3IINFOLTD       3MINDIA  \
0      0  2023-02-20  70.050003  457.812500  35.799999  22085.050781   
1      1  2023-02-21  70.199997  453.237488  35.650002  22212.349609   
2      2  2023-02-22  68.550003  446.312500  35.250000  21809.000000   
3      3  2023-02-23  69.150002  446.387512  34.650002  22091.150391   
4      4  2023-02-24  68.300003  444.174988  34.200001  21853.650391   

      3PLAND      5PAISA     63MOONS  AADHARHFC  ...  ZENSARTECH      ZENTEC  \
0  22.049999  288.200012  203.250000        NaN  ...  271.200012  251.350006   
1  21.350000  285.600006  197.050003        NaN  ...  293.450012  256.750000   
2  20.350000  286.200012  193.550003        NaN  ...  286.100006  247.699997   
3  20.049999  285.899994  194.699997        NaN  ...  300.600006  234.699997   
4  21.049999  282.200012  190.399994        NaN  ...  291.600006  266.950012   

      ZFCVINDIA     ZIMLAB     ZOMATO   

In [4]:
data_frames = cleaned_data_frames

for key in data_frames.keys():
    data_frames[key]['Date'] = pd.to_datetime(data_frames[key]['Date'])
    data_frames[key].set_index('Date', inplace=True)


# Check if the 'Date' column is datetime in each dataframe
for sheet, df in data_frames.items():
    if 'Date' in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df['Date']):
            print(f"'Date' column in {sheet} dataframe is of datetime data type.")
        else:
            print(f"'Date' column in {sheet} dataframe is not of datetime data type.")
    else:
        print(f"'Date' column not found in {sheet} dataframe.")

'Date' column not found in Close dataframe.
'Date' column not found in Open dataframe.
'Date' column not found in High dataframe.
'Date' column not found in Low dataframe.
'Date' column not found in Volume dataframe.


In [5]:
# Function to check for any remaining empty values in cleaned DataFrames
def check_empty_values(data_frames):
    empty_values_info = {}
    for sheet, df in data_frames.items():
        empty_values = df.isnull().sum().sum()  # Count total empty values in the DataFrame
        empty_values_info[sheet] = empty_values
    
    return empty_values_info

# Check for any remaining empty values in cleaned DataFrames
empty_values_info = check_empty_values(data_frames)
print("\nEmpty Values in Cleaned DataFrames:")
for sheet, empty_values in empty_values_info.items():
    print(f"{sheet}: {empty_values}")

# Ensure there are no remaining empty values
for sheet, empty_values in empty_values_info.items():
    assert empty_values == 0, f"{sheet} DataFrame still contains empty values."


Empty Values in Cleaned DataFrames:
Close: 25839
Open: 25839
High: 25839
Low: 25839
Volume: 25839


AssertionError: Close DataFrame still contains empty values.

In [6]:
# Check if all DataFrames have the same index
index_alignment = all(cleaned_data_frames['Close'].index.equals(df.index) for df in cleaned_data_frames.values())

# Check if all DataFrames have the same columns
columns_alignment = all(cleaned_data_frames['Close'].columns.equals(df.columns) for df in cleaned_data_frames.values())

if index_alignment and columns_alignment:
    print("Index and columns alignment match across all DataFrames.")
else:
    print("Index or columns alignment mismatch across DataFrames. DataFrames are not aligned.")

Index and columns alignment match across all DataFrames.


In [34]:
# Function to calculate daily change percentage
def calculate_daily_change(df):
    # Calculate daily changes
    metrics = pd.DataFrame(index=df.index)
    daily_changes = {}
    for stock in df.columns:
        daily_changes[stock + '_Daily_Change'] = ((df[stock] / df[stock].shift(1)) - 1) * 100
    return pd.concat(daily_changes, axis=1)


close_df = data_frames['Close']

daily_change_metrics = calculate_daily_change(close_df)


# Get the last date (t day) in the DataFrame
last_date = daily_change_metrics.index[-1]

# Categorize each stock as a red day or green day
categories = []
for stock in close_df.columns:
    daily_change_column = stock + '_Daily_Change'
    if daily_change_column in daily_change_metrics.columns:
        t_day_change = daily_change_metrics.loc[last_date, daily_change_column]
        if t_day_change > 0:
            categories.append((stock, 'Green Day'))
        else:
            categories.append((stock, 'Red Day'))
    else:
        categories.append((stock, 'Stock Not Found'))

# Create a DataFrame for the categorizations
green_red_day = pd.DataFrame(categories, columns=['Stock', 'T Day Action'])




# Check if 'INFY_Daily_Change' column exists and print the last few values
if 'INFY_Daily_Change' in daily_change_metrics.columns:
    print("\nDaily percentage change for INFY:")
    print(daily_change_metrics['INFY_Daily_Change'].dropna().tail())
else:
    print("\n'INFY' stock not found in the Close DataFrame.")


Daily percentage change for INFY:
Date
2024-06-27    2.119168
2024-06-28   -0.419486
2024-07-01    1.535028
2024-07-02    1.901559
2024-07-03    0.391720
Name: INFY_Daily_Change, dtype: float64


In [9]:
import tabulate

# Function to calculate daily change percentage
def calculate_daily_change(df):
    daily_change = (df / df.shift(1)) - 1
    return daily_change

# Calculate the daily change percentage for Close prices
close_df = data_frames['Close']
daily_change_close = calculate_daily_change(close_df)

def calculate_fibonacci_levels(stock,tolerance = 0.015):   #We are at 1.5% tolerance
    high_df = data_frames['High']
    low_df = data_frames['Low']
    open_df = data_frames['Open']
    close_df = data_frames['Close']
    
    highest_high = high_df[stock].max()
    lowest_low = low_df[stock].min()
    range_of_movement = highest_high - lowest_low

    fib_levels = {
        '0%': highest_high,
        '9.01%': highest_high - 0.0901 * range_of_movement,
        '14.6%': highest_high - 0.146 * range_of_movement,
        '23.6%': highest_high - 0.236 * range_of_movement,
        '38.2%': highest_high - 0.382 * range_of_movement,
        '50%': highest_high - .5 * range_of_movement,
        '61.8%': highest_high - .618 * range_of_movement,
        '78.6%': highest_high - .786 * range_of_movement,
        '85.79%': highest_high - .8579 * range_of_movement,
        '100%': lowest_low
    }

    bounds = {}
    for level_name, level_price in fib_levels.items():
        lower_bound = level_price - tolerance * range_of_movement
        upper_bound = level_price + tolerance * range_of_movement
        bounds[level_name] = (lower_bound, upper_bound)

    return bounds

def categorize_stock(stock, bounds):
    open_df = data_frames['Open']
    close_df = data_frames['Close']
    
    last_date = close_df.index[-1]
    t_day_open = open_df.loc[last_date, stock]
    t_day_close = close_df.loc[last_date, stock]
    t_day_green = t_day_close > t_day_open

    for level_name, (lower_bound, upper_bound) in bounds.items():
        if t_day_green:
            if lower_bound <= t_day_open <= upper_bound or lower_bound <= t_day_close <= upper_bound:                     #Retain the option of "or" 
                return level_name
        else:
            if lower_bound <= t_day_close <= upper_bound:
                return level_name

    return "Outside tolerance"


# Apply the function to categorize each stock
categorized_stocks = {}

for stock in data_frames['Close'].columns:
    bounds = calculate_fibonacci_levels(stock)
    category = categorize_stock(stock, bounds)
    categorized_stocks[stock] = category

# Convert the dictionary to a DataFrame for better visualization
fibonacci_retracement_metrics = pd.DataFrame.from_dict(categorized_stocks, orient='index', columns=['Category'])
#print(fibonacci_retracement_metrics)


filtered_stocks_df = fibonacci_retracement_metrics[fibonacci_retracement_metrics['Category'] != 'Outside tolerance']
#filtered_stocks_sorted = filtered_stocks_df[filtered_stocks_df['Category'].isin(['38.2%'])]
print(filtered_stocks_df)

           Category
20MICRONS     9.01%
3MINDIA       9.01%
AAKASH           0%
AARTIPHARM    9.01%
AARVI           50%
...             ...
ZENITHSTL        0%
ZENTEC           0%
ZOTA          14.6%
ZUARI         23.6%
ZYDUSWELL     23.6%

[651 rows x 1 columns]


In [17]:
def check_fibonacci_level(stock_symbol):
    if stock_symbol in categorized_stocks:
        return categorized_stocks[stock_symbol]
    else:
        return "Stock symbol not found in categorized stocks."

# Ask for user input
stock_symbol = input("Enter the ticker symbol of the stock to check Fibonacci level: ")

# Call the function to check Fibonacci level
fibonacci_level = check_fibonacci_level(stock_symbol.upper())  # Convert to uppercase to handle case sensitivity
print(f"Fibonacci level for {stock_symbol}: {fibonacci_level}")

Fibonacci level for ASIANPAINT: Outside tolerance


In [10]:
# Calculate money flow multiplier for each stock and store in a DataFrame
money_flow_multiplier_metrics = pd.DataFrame(index=close_df.columns, columns=['Money_Flow_Multiplier'])

def calculate_money_flow_multiplier(high, low, close):
    money_flow_raw = ((close - low) - (high - close))
    money_flow_volume = high - low
    money_flow_multiplier = money_flow_raw / money_flow_volume
    return money_flow_multiplier

for stock in close_df.columns:
    high = data_frames['High'].loc[data_frames['High'].index[-1], stock]
    low = data_frames['Low'].loc[data_frames['Low'].index[-1], stock]
    close = data_frames['Close'].loc[data_frames['Close'].index[-1], stock]
    
    money_flow_multiplier_metrics.loc[stock, 'Money_Flow_Multiplier'] = calculate_money_flow_multiplier(high, low, close)

for stock in close_df.columns:
    volume_t = data_frames['Volume'].loc[data_frames['Volume'].index[-1], stock]
    close_t = data_frames['Close'].loc[data_frames['Close'].index[-1], stock]
    tam_in_lakhs = (volume_t * close_t) / 10**5
    
    # Append TAM and other columns to the money flow multiplier table
    money_flow_multiplier_metrics.loc[stock, 'Volume'] = volume_t
    money_flow_multiplier_metrics.loc[stock, 'Close'] = close_t
    money_flow_multiplier_metrics.loc[stock, 'TAM in Lakhs'] = tam_in_lakhs


# Filter the DataFrame for stocks with money flow multiplier between 0.9 and 1
high_multiplier_stocks = money_flow_multiplier_metrics[
    (money_flow_multiplier_metrics['Money_Flow_Multiplier'] >= 0.9) &
    (money_flow_multiplier_metrics['Money_Flow_Multiplier'] <= 1)
]

# Filter out rows where 'TAM in Lakhs' is less than 500
high_multiplier_stocks_filtered = high_multiplier_stocks[high_multiplier_stocks['TAM in Lakhs'] >= 500]
high_multiplier_stocks_sorted = high_multiplier_stocks_filtered.sort_values(by=['Money_Flow_Multiplier', 'Volume', 'TAM in Lakhs'],ascending=False)

print("Fringe buys = Stocks with money flow multiplier between 0.9 and 1:")
print(high_multiplier_stocks_sorted.to_markdown(index=True, tablefmt="grid"))

C:\Users\dbcin\AppData\Local\Temp\ipykernel_17552\1886015577.py:7: RuntimeWarning: invalid value encountered in scalar divide
  money_flow_multiplier = money_flow_raw / money_flow_volume


Fringe buys = Stocks with money flow multiplier between 0.9 and 1:
+------------+-------------------------+------------------+---------+----------------+
|            |   Money_Flow_Multiplier |           Volume |   Close |   TAM in Lakhs |
+============+=========================+==================+=========+================+
| GTLINFRA   |                1        |      1.20665e+08 |    3.76 |       4536.99  |
+------------+-------------------------+------------------+---------+----------------+
| KBCGLOBAL  |                1        |      4.38606e+07 |    2.01 |        881.598 |
+------------+-------------------------+------------------+---------+----------------+
| SARVESHWAR |                1        |      6.1127e+06  |    9.83 |        600.879 |
+------------+-------------------------+------------------+---------+----------------+
| CCHHL      |                1        |      3.33558e+06 |   24.64 |        821.887 |
+------------+-------------------------+------------------+----

In [11]:
volume_stratifier = pd.DataFrame(index=data_frames['Close'].columns, columns=['TAM in lakhs'])

# Function to calculate the TAM in lakhs
def calculate_tam(stock):
    close_df = data_frames['Close']
    volume_df = data_frames['Volume']
    
    last_date = close_df.index[-1]

    t_day_close = close_df.loc[last_date, stock]
    t_day_volume = volume_df.loc[last_date, stock]

    # Calculate TAM in lakhs
    tam = (t_day_volume * t_day_close) / 10**5
    return tam

# Apply the function to calculate TAM for each stock
for stock in data_frames['Close'].columns:
    tam = calculate_tam(stock)
    volume_stratifier.loc[stock, 'TAM in lakhs'] = tam

print("Volume Stratifier")
print(volume_stratifier)

Volume Stratifier
           TAM in lakhs
20MICRONS   1359.350958
360ONE      3310.626353
3IINFOLTD   1472.650518
3MINDIA     7273.851324
3PLAND        14.183292
...                 ...
ZOTA         291.077325
ZUARI       1436.870416
ZUARIIND    1824.384506
ZYDUSLIFE  29294.060315
ZYDUSWELL   5684.969307

[1706 rows x 1 columns]


In [12]:
appreciation_status = pd.DataFrame(index=data_frames['Close'].columns, columns=['Appreciation Status'])

# Function to determine stock appreciation/depreciation status
def determine_appreciation_status(stock):
    close_df = data_frames['Close']
    
    first_day_close = close_df.iloc[0, close_df.columns.get_loc(stock)]
    last_day_close = close_df.iloc[-1, close_df.columns.get_loc(stock)]
    
    if first_day_close > last_day_close:
        return "Depreciating stock"
    else:
        return "Appreciating stock"

# Apply the function to determine status for each stock
for stock in data_frames['Close'].columns:
    appreciation_status.loc[stock, 'Appreciation Status'] = determine_appreciation_status(stock)


""""
while True:
   ticker_name = input("Enter ticker name (or 'exit' to quit): ").strip().upper()
   
   if ticker_name == 'EXIT':
       break
   
   if ticker_name in appreciation_status.index:
       status = appreciation_status.loc[ticker_name, 'Appreciation Status']
       print(f"Appreciation status for {ticker_name}: {status}")
   else:
      print(f"Ticker '{ticker_name}' not found.")

"""

'"\nwhile True:\n   ticker_name = input("Enter ticker name (or \'exit\' to quit): ").strip().upper()\n   \n   if ticker_name == \'EXIT\':\n       break\n   \n   if ticker_name in appreciation_status.index:\n       status = appreciation_status.loc[ticker_name, \'Appreciation Status\']\n       print(f"Appreciation status for {ticker_name}: {status}")\n   else:\n      print(f"Ticker \'{ticker_name}\' not found.")\n\n'

In [15]:
def calculate_smi(data_frames, k_period=14, d_period=14, smoothing_period=3, use_sma_for_d=False):
    high_df = data_frames['High']
    low_df = data_frames['Low']
    open_df = data_frames['Open']
    close_df = data_frames['Close']

   
    smi_data = []  # List to store data for smi_df construction

    for stock in close_df.columns:
        # Calculate Center of High/Low Range (C)
        high_max = high_df[stock].rolling(window=k_period).max()
        low_min = low_df[stock].rolling(window=k_period).min()
        center_range = (high_max + low_min) / 2
        
        # Calculate H = CC TODAY - C
        current_close = close_df[stock]
        H = current_close - center_range
        
        # Smooth H using a 3-period Exponential Moving Average (HS1)
        HS1 = H.ewm(span=smoothing_period, min_periods=smoothing_period).mean() * 3
        HS2 = HS1.ewm(span=smoothing_period, min_periods=smoothing_period).mean() * 3
        
        # Calculate High-Low Range (DHL)
        DHL = (high_max - low_min)
        
        # Smooth DHL using a 3-period Exponential Moving Average (DHL1)
        DHL1 = DHL.ewm(span=smoothing_period, min_periods=smoothing_period).mean() * 3
        DHL2 = DHL1.ewm(span=smoothing_period, min_periods=smoothing_period).mean() * 3/2
        
        # Calculate %K and %D
        K_smoothed = 100 * HS2 / DHL2
        
        # Calculate %D using either EMA or SMA
        if use_sma_for_d:
            D_smoothed = K_smoothed.rolling(window=d_period, min_periods=d_period).mean()
        else:
            D_smoothed = K_smoothed.ewm(span=d_period, min_periods=d_period).mean()



        diff_smoothed = K_smoothed - D_smoothed
        
        
        # Store %K and %D values in smi_data
        smi_data.append(K_smoothed.rename(f'%K_{stock}'))
        smi_data.append(D_smoothed.rename(f'%D_{stock}'))
        smi_data.append(diff_smoothed.rename(f'diff_{stock}'))

    # Concatenate all data into smi_df
    smi_df = pd.concat(smi_data, axis=1)

    return smi_df

smi_df = calculate_smi(data_frames, use_sma_for_d=False)

In [16]:
#Just a check
stock_symbol = input("Enter the stock ticker to view %K and %D values (e.g., INFY): ").strip().upper()

# Check if the stock symbol exists in the dataframe
if f'%K_{stock_symbol}' in smi_df.columns and f'%D_{stock_symbol}' in smi_df.columns:
    # Print %K and %D for the specified stock symbol
    print(f"\n%K for {stock_symbol}:")
    print(smi_df[f'%K_{stock_symbol}'].tail())

    print(f"\n%D for {stock_symbol}:")
    print(smi_df[f'%D_{stock_symbol}'].tail())

    print(f"Difference for {stock_symbol}:")
    print(smi_df[f'diff_{stock_symbol}'].tail())
else:
    print(f"Stock symbol '{stock_symbol}' not found in the dataframe.")


%K for ASIANPAINT:
Date
2024-06-27   -36.329804
2024-06-28   -24.519820
2024-07-01    -1.685542
2024-07-02    20.825447
2024-07-03    35.712062
Name: %K_ASIANPAINT, dtype: float64

%D for ASIANPAINT:
Date
2024-06-27   -1.660267
2024-06-28   -4.708207
2024-07-01   -4.305185
2024-07-02   -0.954434
2024-07-03    3.934432
Name: %D_ASIANPAINT, dtype: float64
Difference for ASIANPAINT:
Date
2024-06-27   -34.669537
2024-06-28   -19.811613
2024-07-01     2.619644
2024-07-02    21.779881
2024-07-03    31.777630
Name: diff_ASIANPAINT, dtype: float64


In [54]:
def categorize_stocks(smi_df):
    categorization = pd.DataFrame(index=smi_df.index)

    for stock in smi_df.columns:
        if stock.startswith('%K_'):
            stock_ticker = stock.replace('%K_', '')
            k_column = f'%K_{stock_ticker}'
            d_column = f'%D_{stock_ticker}'

            # Check if %K smoothed is lower than %D smoothed
            k_smoothed = smi_df[k_column]
            d_smoothed = smi_df[d_column]
            is_k_below_d = k_smoothed < d_smoothed

            # Define all categories upfront
            all_categories = ['-100,-40', '-40-0', '>0', 'Not Below %D']
            
            # Categorize based on %K smoothed value
            labels = pd.cut(
                k_smoothed,
                bins=[-float('inf'), -40, 0, float('inf')],
                labels=all_categories[:-1]  # Exclude 'Not Below %D'
            ).astype(pd.api.types.CategoricalDtype(categories=all_categories))
            
            # Create a mask for %K smoothed being below %D smoothed
            labels = labels.where(is_k_below_d, other='Not Below %D')


            categorization[f'Category_{stock_ticker}'] = labels

    return categorization

categorization_df = categorize_stocks(smi_df)

In [55]:
def check_decreasing_trend(diff_values, ksmoothed_values, lookback_period, max_final_diff=25):   
    """
    Checks if the absolute value of the negative difference (%K - %D) is decreasing over a flexible lookback period
    and if %K smoothed values are increasing over the same period.
    
    Parameters:
    diff_values (list or array-like): List of difference values (%K - %D) over time.
    ksmoothed_values (list or array-like): List of %K smoothed values over time.
    lookback_period (int): Number of previous days to check the trend.
    max_final_diff (int): Maximum allowed absolute difference on the final day (t day).

    Returns:
    bool: True if the absolute negative difference is decreasing and %K smoothed values are increasing
          over the lookback period, False otherwise.
    """
    if len(diff_values) < lookback_period + 1 or len(ksmoothed_values) < lookback_period + 1:
        return False
    
    # Ensure all differences in the lookback period are negative
    for i in range(lookback_period + 1):
        if diff_values[i] > 0:
            return False
    
    # Check if the absolute value of the differences is decreasing
    for i in range(lookback_period):
        if abs(diff_values[i]) < abs(diff_values[i + 1]):
            return False

    # # Check if %K smoothed values are increasing over the lookback period
    # for i in range(lookback_period):
    #     if ksmoothed_values[i] > ksmoothed_values[i + 1]:
    #         return False
    
    # Check the difference on the final day (t day)
    final_diff = abs(diff_values[-1])
    if final_diff > max_final_diff:
        return False
    
    return True

""" This is the code implementation for SMI convergence and spiking """

def identify_convergence(smi_df, categorization_df, lookback_period=2):
    convergence_results = []

    # Get the last n+1 rows (including t and the previous days)
    last_n_days = smi_df.tail(lookback_period + 1)
    print(f"Using data for the last {lookback_period + 1} days: {last_n_days.index.tolist()}")
    
    for stock in smi_df.columns:
        if stock.startswith('%K_'):
            stock_ticker = stock.replace('%K_', '')
            category_col = f'Category_{stock_ticker}'

            # Check if the stock falls into a bucket other than 'Not Below %D'
            if category_col in categorization_df.columns:
                last_category = categorization_df[category_col].iloc[-1]

                if last_category != 'xx':
                    # Get %K and %D smoothed values for the last n+1 days
                    k_smoothed_values = last_n_days[f'%K_{stock_ticker}']
                    d_smoothed_values = last_n_days[f'%D_{stock_ticker}']

                    # Calculate differences for the last n+1 days
                    diff_values = k_smoothed_values - d_smoothed_values

                    # Check if the differences are strictly decreasing
                    is_decreasing = check_decreasing_trend(diff_values, k_smoothed_values, lookback_period)

                    # Identify convergence spikes and short options
                    if diff_values.iloc[-2] < 0 and diff_values.iloc[-1] > 0:
                        convergence_spike = True
                    elif diff_values.iloc[-3] < 0 and diff_values.iloc[-1] > 0 and k_smoothed_values.iloc[-1] > k_smoothed_values.iloc[-2]:
                        convergence_spike = True
                    else:
                        convergence_spike = False
                    
                    if diff_values.iloc[-2] > 0 and diff_values.iloc[-1] < 0:
                        short_option = True
                    else:
                        short_option = False

                    # Append result to convergence_results as a tuple
                    convergence_results.append((stock_ticker, last_category, is_decreasing, convergence_spike, short_option))

    # Create DataFrame from convergence_results
    columns = ['Stock', 'Category', 'Is_Converging', 'Convergence_Spike', 'Short_Option']
    convergence_df = pd.DataFrame(convergence_results, columns=columns)

    return convergence_df

# Example usage
lookback_period = 1  # Adjust this to change the lookback period

convergence_df = identify_convergence(smi_df, categorization_df, lookback_period)

Using data for the last 3 days: [Timestamp('2024-07-01 00:00:00'), Timestamp('2024-07-02 00:00:00'), Timestamp('2024-07-03 00:00:00')]


C:\Users\dbcin\AppData\Local\Temp\ipykernel_17552\2477939740.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if diff_values[i] > 0:
C:\Users\dbcin\AppData\Local\Temp\ipykernel_17552\2477939740.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if abs(diff_values[i]) < abs(diff_values[i + 1]):
C:\Users\dbcin\AppData\Local\Temp\ipykernel_17552\2477939740.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  final_diff = abs(diff

In [58]:
# Filter for Is_Converging stocks
is_converging_df = convergence_df[convergence_df['Is_Converging'] == True]

# Define the order of categories
category_order = {
    '-100,-40': 1,
    '-40-0': 2,
    '> 0': 3,
    'Not Below %D': 4
}

# Function to get the category of a stock
def get_category(stock, categorization_df):
    category_col = f'Category_{stock}'
    if category_col in categorization_df.columns:
        return categorization_df[category_col].iloc[-1]
    return None

# Create the ordered DataFrame
def create_ordered_converging_df(is_converging_df, categorization_df, category_order):
    result = []
    for _, row in is_converging_df.iterrows():
        stock = row['Stock']
        category = get_category(stock, categorization_df)
        result.append((stock, category))
    
    # Convert to DataFrame
    result_df = pd.DataFrame(result, columns=['Stock', 'Category'])
    
    # Sort by category order
    result_df['Category_Order'] = result_df['Category'].map(category_order)
    result_df.sort_values(by=['Category_Order', 'Stock'], inplace=True)
    result_df.drop(columns=['Category_Order'], inplace=True)
    
    return result_df

# Create ordered DataFrame for Is_Converging stocks
ordered_is_converging_df = create_ordered_converging_df(is_converging_df, categorization_df, category_order)

# Display the result
print("Ordered Table for Is_Converging Stocks:")
print(ordered_is_converging_df)


# Function to get the category of a stock on t-1 day
def get_category_t_minus_1(stock, categorization_df):
    category_col = f'Category_{stock}'
    if category_col in categorization_df.columns:
        return categorization_df[category_col].iloc[-2]  # t-1 day
    return None

# Create the ordered DataFrame for Convergence Spike stocks
def create_ordered_spike_df(convergence_spike_df, categorization_df, category_order):
    result = []
    for _, row in convergence_spike_df.iterrows():
        stock = row['Stock']
        category = get_category_t_minus_1(stock, categorization_df)
        result.append((stock, category))
    
    # Convert to DataFrame
    result_df = pd.DataFrame(result, columns=['Stock', 'Category'])
    
    # Sort by category order
    result_df['Category_Order'] = result_df['Category'].map(category_order)
    result_df.sort_values(by=['Category_Order', 'Stock'], inplace=True)
    result_df.drop(columns=['Category_Order'], inplace=True)
    
    return result_df

# Example usage
# Filter for Convergence Spike stocks
convergence_spike_df = convergence_df[convergence_df['Convergence_Spike'] == True]
ordered_convergence_spike_df = create_ordered_spike_df(convergence_spike_df, categorization_df, category_order)

# Display the result
print("Ordered Table for Convergence Spike Stocks:")
print(ordered_convergence_spike_df)

Ordered Table for Is_Converging Stocks:
        Stock  Category
6     AIROLAM  -100,-40
12       ATGL  -100,-40
18        AWL  -100,-40
19    BAIDFIN  -100,-40
27   BEDMUTHA  -100,-40
..        ...       ...
179    UTIAMC        >0
181      VEDL        >0
182   VENKEYS        >0
184     VPRPL        >0
186     WORTH        >0

[189 rows x 2 columns]
Ordered Table for Convergence Spike Stocks:
         Stock  Category
19       BSHSL  -100,-40
77    KARMAENG  -100,-40
86   LANCORHOL  -100,-40
4     ACEINTEG     -40-0
7        AKASH     -40-0
..         ...       ...
147   TVSELECT        >0
148       UCAL        >0
150   UNOMINDA        >0
155     WELENT        >0
156     ZIMLAB        >0

[158 rows x 2 columns]


In [23]:
# Get unique stocks from each dataframe
is_converging_stocks = set(ordered_is_converging_df['Stock'])
convergence_spike_stocks = set(ordered_convergence_spike_df['Stock'])

# Get the union of the two sets
unique_stocks = is_converging_stocks.union(convergence_spike_stocks)

# Convert to a sorted list
unique_stocks_list_cc = sorted(unique_stocks)

# Display the result
print("List of Unique Stocks in Is_Converging and Convergence Spikes:")
print(unique_stocks_list_cc)

List of Unique Stocks in Is_Converging and Convergence Spikes:
['AADHARHFC', 'AARTIDRUGS', 'AARTIIND', 'AARVEEDEN', 'ABBOTINDIA', 'ABDL', 'ABFRL', 'ACC', 'ACEINTEG', 'ACI', 'ADANIENSOL', 'AEGISLOG', 'AIAENG', 'AIROLAM', 'AKASH', 'ALLCARGO', 'ALPHAGEO', 'AMBIKCO', 'AMBUJACEM', 'ANANTRAJ', 'ANTGRAPHIC', 'APLAPOLLO', 'APTUS', 'ARIHANTCAP', 'ARROWGREEN', 'ARVIND', 'ATFL', 'ATGL', 'AUROPHARMA', 'AUTOAXLES', 'AVROIND', 'AVTNPL', 'AWFIS', 'AWL', 'AXISCADES', 'BAIDFIN', 'BAJAJHLDNG', 'BALAMINES', 'BALKRISHNA', 'BALPHARMA', 'BALUFORGE', 'BARBEQUE', 'BATAINDIA', 'BBL', 'BEDMUTHA', 'BIRLACABLE', 'BLUEDART', 'BOROLTD', 'BOROSCI', 'BSHSL', 'CAMPUS', 'CAMS', 'CAPITALSFB', 'CARERATING', 'CARYSIL', 'CASTROLIND', 'CCHHL', 'CELLO', 'CENTURYTEX', 'CERA', 'CGPOWER', 'CHEMCON', 'CIEINDIA', 'CIGNITITEC', 'COALINDIA', 'COCHINSHIP', 'COLPAL', 'CONCORDBIO', 'CORALFINAC', 'CSLFINANCE', 'DABUR', 'DALBHARAT', 'DAVANGERE', 'DBREALTY', 'DEEDEV', 'DEEPAKFERT', 'DEN', 'DENORA', 'DEVIT', 'DHARMAJ', 'DIAMINESQ', 'DLINK

In [25]:
money_flow_multiplier_metrics = check_and_reindex(fibonacci_retracement_metrics, money_flow_multiplier_metrics)
volume_stratifier = check_and_reindex(fibonacci_retracement_metrics,volume_stratifier)
appreciation_status = check_and_reindex(volume_stratifier,appreciation_status)

The indices of the DataFrames are the same.
The indices of the DataFrames are the same.
The indices of the DataFrames are the same.


In [60]:
import pandas as pd

# Concatenate all data into one DataFrame
merged_data = pd.concat([appreciation_status, volume_stratifier, fibonacci_retracement_metrics, money_flow_multiplier_metrics], axis=1)

# Define criteria for filtering
criteria = (
    (merged_data['TAM in lakhs'] > 500) &  # Volume > 500 lakhs
    (merged_data['Category'] != 'Outside tolerance')  # Not labeled as "Outside tolerance"
)

# Apply filtering based on criteria
filtered_data = merged_data[criteria]

# Define sorting order
sort_order = ['Category', 'Money_Flow_Multiplier', 'TAM in lakhs']

# Sort filtered data
sorted_data = filtered_data.sort_values(by=sort_order, ascending=[True, False, False])

# Select columns to display
columns_to_display = ['TAM in lakhs', 'Money_Flow_Multiplier', 'Category', 'Close']

# Display sorted and filtered data
print("Sorted and Filtered Data:")
print(sorted_data[columns_to_display].to_markdown(index=True, tablefmt="grid"))

Sorted and Filtered Data:
+------------+----------------+-------------------------+------------+----------+
|            |   TAM in lakhs |   Money_Flow_Multiplier | Category   |    Close |
+============+================+=========================+============+==========+
| ZENTEC     |      21807.9   |              1          | 0%         |  1429.8  |
+------------+----------------+-------------------------+------------+----------+
| KINGFA     |       6542.71  |              1          | 0%         |  2604.5  |
+------------+----------------+-------------------------+------------+----------+
| GENUSPOWER |       4613.89  |              1          | 0%         |   357.4  |
+------------+----------------+-------------------------+------------+----------+
| GTLINFRA   |       4536.99  |              1          | 0%         |     3.76 |
+------------+----------------+-------------------------+------------+----------+
| DREDGECORP |       3308.37  |              1          | 0%         |  

In [61]:
# Assuming sorted_data is already defined and contains the sorted and filtered DataFrame
unique_stocks_above = sorted_data.index.unique().tolist()
print(unique_stocks_above)

common_stocks = list(set(unique_stocks_list_cc).intersection(set(unique_stocks_above)))
print(common_stocks)

print("\nColumns in categories_df:")
print(green_red_day.columns)

['ZENTEC', 'KINGFA', 'GENUSPOWER', 'GTLINFRA', 'DREDGECORP', 'THEINVEST', 'WALCHANNAG', 'CCHHL', 'INDOWIND', 'ONEPOINT', 'SRGHFL', 'INDUSTOWER', 'OFSS', 'HUDCO', 'PRECWIRE', 'BAJAJHLDNG', 'MAZDOCK', 'RKFORGE', 'PITTIENG', 'SIGNATURE', 'COCHINSHIP', 'GRSE', 'GRINDWELL', 'SHYAMMETL', 'SUNTV', 'SUNDRMFAST', 'VGUARD', 'CIGNITITEC', 'GODREJPROP', 'JSWSTEEL', 'PERSISTENT', 'ICICIGI', 'GLENMARK', 'MOTHERSON', 'BOMDYEING', 'HILTON', 'GMRP&UI', 'SUNPHARMA', 'SWELECTES', 'STCINDIA', 'SOTL', 'GPPL', 'SUNDARMFIN', 'KPIL', 'EICHERMOT', 'POLYCAB', 'SHREEPUSHK', 'KSCL', 'VEDL', 'PRAKASH', 'FORTIS', 'COSMOFIRST', 'SGIL', 'INDIACEM', 'HFCL', 'ENGINERSIN', 'DATAPATTNS', 'PIDILITIND', 'IDEA', 'ABDL', 'IRFC', 'HCC', 'TORNTPOWER', 'FINPIPE', 'DIAMINESQ', 'FDC', 'COALINDIA', 'GRINFRA', 'HOMEFIRST', 'ARVINDFASN', 'BHARATFORG', 'PGIL', 'NAVKARCORP', 'AVANTIFEED', 'AGARIND', 'UNIVCABLES', 'GODREJIND', 'SBILIFE', 'ARROWGREEN', 'BHARTIARTL', 'BECTORFOOD', 'RATNAMANI', 'GIPCL', 'MARKSANS', 'ASHAPURMIN', 'SONACOMS

In [62]:
# Convert selected_stocks set to a list
selected_stocks = list(common_stocks)

# Create a dataframe to store the selected data
selected_data = pd.DataFrame(index=selected_stocks)

#columns_to_select = ['TAM in lakhs', 'Money_Flow_Multiplier', 'Category', 'Close']

# Check and annotate which stocks are in ordered_is_converging_df and get their bucket
selected_data['Is_Converging'] = selected_data.index.isin(ordered_is_converging_df['Stock'])
selected_data['Is_Converging_Bucket'] = selected_data.apply(lambda x: ordered_is_converging_df.loc[ordered_is_converging_df['Stock'] == x.name, 'Category'].iloc[-1] if x['Is_Converging'] else None, axis=1)

# Merge with sorted_data to get TAM in lakhs, Money_Flow_Multiplier, Category, Close for selected stocks
selected_data = selected_data.merge(sorted_data[columns_to_display], left_index=True, right_index=True, how='left')


# Merge with categories_df to get Green Day / Red Day information
selected_data = selected_data.merge(green_red_day, left_index=True, right_on='Stock', how='left')

# Reset the index to have 'Stock' as a regular column
selected_data.reset_index(inplace=True)
selected_data.rename(columns={'index': 'Ticker Symbol'}, inplace=True)  # Rename the index column to 'Stock'


bucket_order = ['-100,-40', '-40-0', '>0', 'Not Below %D']

criteria = (
    (selected_data['TAM in lakhs'] > 500) & # Volume > 500 lakhs
    (selected_data['Is_Converging']) 
)

# Set 'Is_Converging_Bucket' as a categorical column with the specified order
selected_data['Is_Converging_Bucket'] = pd.Categorical(selected_data['Is_Converging_Bucket'], categories=bucket_order, ordered=True)

category_order = ['0%', '9.01%', '14.6%', '23.6%', '38.2%', '50%', '61.8%', '78.6%',  '85.79%', '100%']


# Set 'Category' as a categorical column with the specified order
selected_data['Category'] = pd.Categorical(selected_data['Category'], categories=category_order, ordered=True)

# Sort the DataFrame by 'Is_Converging_Bucket'
selected_data = selected_data.sort_values(by=['Category','Is_Converging_Bucket','Money_Flow_Multiplier'],ascending=[False,True,False])

selected_data = selected_data[criteria]

# Reorder columns to have 'Stock' as the first column
selected_data = selected_data[['Stock'] + [col for col in selected_data.columns if col != 'Stock' and col != 'Ticker Symbol']]

# Display the selected data
print("Selected Data for Listed Stocks with Convergence Status and Buckets:")
print(selected_data.to_markdown(index=False, tablefmt="grid"))

Selected Data for Listed Stocks with Convergence Status and Buckets:
+------------+-----------------+------------------------+----------------+-------------------------+------------+----------+----------------+
| Stock      | Is_Converging   | Is_Converging_Bucket   |   TAM in lakhs |   Money_Flow_Multiplier | Category   |    Close | T Day Action   |
+============+=================+========================+================+=========================+============+==========+================+
| HILTON     | True            | -100,-40               |       2081.15  |              -0.360239  | 100%       |    80.05 | Red Day        |
+------------+-----------------+------------------------+----------------+-------------------------+------------+----------+----------------+
| AWL        | True            | -100,-40               |       2766.05  |              -0.662341  | 78.6%      |   335.65 | Red Day        |
+------------+-----------------+------------------------+----------------+-----

C:\Users\dbcin\AppData\Local\Temp\ipykernel_17552\3920351876.py:44: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  selected_data = selected_data[criteria]


In [63]:
# Convert selected_stocks set to a list
selected_stocks = list(common_stocks)


# Create a dataframe to store the selected data
selected_data_spike = pd.DataFrame(index=selected_stocks)


# Check and annotate which stocks are in ordered_convergence_spikes_df and get their bucket
selected_data_spike['Convergence_Spikes'] = selected_data_spike.index.isin(ordered_convergence_spike_df['Stock'])
selected_data_spike['Convergence_Spikes_Bucket'] = selected_data_spike.apply(lambda x: ordered_convergence_spike_df.loc[ordered_convergence_spike_df['Stock'] == x.name, 'Category'].iloc[-1] if x['Convergence_Spikes'] else None, axis=1)

# Merge with sorted_data to get TAM in lakhs, Money_Flow_Multiplier, Category, Close for selected stocks
selected_data_spike = selected_data_spike.merge(sorted_data[columns_to_display], left_index=True, right_index=True, how='left')

# Merge with categories_df to get Green Day / Red Day information
selected_data_spike = selected_data_spike.merge(green_red_day, left_index=True, right_on='Stock', how='left')


# Reset the index to have 'Stock' as a regular column
selected_data_spike.reset_index(inplace=True)
selected_data_spike.rename(columns={'index': 'Ticker Symbol'}, inplace=True)  # Rename the index column to 'Stock'

bucket_order = ['-100,-40', '-40-0', '>0', 'Not Below %D']

criteria = (
    (selected_data_spike['TAM in lakhs'] > 500) & # Volume > 500 lakhs
    (selected_data_spike['Convergence_Spikes']) 
)

# Set 'Is_Converging_Bucket' as a categorical column with the specified order
selected_data_spike['Convergence_Spikes_Bucket'] = pd.Categorical(selected_data_spike['Convergence_Spikes_Bucket'], categories=bucket_order, ordered=True)

category_order = ['0%', '9.01%', '14.6%', '23.6%', '38.2%', '50%', '61.8%', '78.6%',  '85.79%', '100%']


# Set 'Category' as a categorical column with the specified order
selected_data_spike['Category'] = pd.Categorical(selected_data_spike['Category'], categories=category_order, ordered=True)


# Sort the DataFrame by 'Is_Converging_Bucket'
selected_data_spike = selected_data_spike.sort_values(by=['Category','Convergence_Spikes_Bucket','Money_Flow_Multiplier'],ascending=[False,True,False])

selected_data_spike = selected_data_spike[criteria]


# Reorder columns to have 'Stock' as the first column
selected_data_spike = selected_data_spike[['Stock'] + [col for col in selected_data_spike.columns if col != 'Stock' and col != 'Ticker Symbol']]

# Display the selected data
print("Selected Data for Listed Stocks with Convergence Spikes:")
print(selected_data_spike.to_markdown(index=False, tablefmt="grid"))

Selected Data for Listed Stocks with Convergence Spikes:
+------------+----------------------+-----------------------------+----------------+-------------------------+------------+---------+----------------+
| Stock      | Convergence_Spikes   | Convergence_Spikes_Bucket   |   TAM in lakhs |   Money_Flow_Multiplier | Category   |   Close | T Day Action   |
+============+======================+=============================+================+=========================+============+=========+================+
| BSHSL      | True                 | -100,-40                    |        817.806 |               0.80208   | 85.79%     |  217.96 | Green Day      |
+------------+----------------------+-----------------------------+----------------+-------------------------+------------+---------+----------------+
| KRBL       | True                 | -40-0                       |       8217.42  |               0.630633  | 78.6%      |  311.95 | Green Day      |
+------------+----------------------+

C:\Users\dbcin\AppData\Local\Temp\ipykernel_17552\1842112233.py:44: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  selected_data_spike = selected_data_spike[criteria]


In [47]:
"""
From here on out, the code will show calculations for money_flow - 

Bear that in mind while proceeding with analysis

"""

# Get unique stocks from each dataframe
is_converging_stocks = set(ordered_is_converging_df['Stock'])
convergence_spike_stocks = set(ordered_convergence_spike_df['Stock'])

# Assuming money_flow_multiplier_metrics is already defined and populated
# Extract the stock names from the index of the DataFrame
stock_names_mf = money_flow_multiplier_metrics.index.tolist()


isconverginglist = list(set(is_converging_stocks).intersection(set(stock_names_mf)))

convergence_spike_list = list(set(convergence_spike_stocks).intersection(set(stock_names_mf)))

In [49]:
money_table = money_flow_multiplier_metrics

criteria = (
    (money_flow_multiplier_metrics['TAM in Lakhs'] > 500) &  # Volume > 500 lakhs
    (money_flow_multiplier_metrics['Money_Flow_Multiplier'] > 0.5)  # "Above 0.5"
)

filtered_money_mf = money_table[criteria]

sort_order = ['TAM in Lakhs','Money_Flow_Multiplier']

sorted_filtered_money_mf_data = filtered_money_mf.sort_values(by=sort_order, ascending=[False, False])

print("Sorted and Filtered Money Multiplier Data:")
print(sorted_filtered_money_mf_data.to_markdown(index=True, tablefmt="grid"))

Sorted and Filtered Money Multiplier Data:
+------------+-------------------------+------------------+----------+----------------+
|            |   Money_Flow_Multiplier |           Volume |    Close |   TAM in Lakhs |
+============+=========================+==================+==========+================+
| MAZDOCK    |                0.784384 |      9.1786e+06  |  4684.55 |     429976     |
+------------+-------------------------+------------------+----------+----------------+
| IREDA      |                0.67659  |      1.59629e+08 |   223.11 |     356148     |
+------------+-------------------------+------------------+----------+----------------+
| CASTROLIND |                0.527428 |      1.11256e+08 |   243.17 |     270542     |
+------------+-------------------------+------------------+----------+----------------+
| COCHINSHIP |                0.753925 |      1.05687e+07 |  2436.35 |     257492     |
+------------+-------------------------+------------------+----------+-------

In [64]:
# This is the moneyed is_converging data

money_data = pd.DataFrame(index=isconverginglist)
columns_to_display = ['TAM in Lakhs', 'Money_Flow_Multiplier', 'Close']

# Check and annotate which stocks are in ordered_is_converging_df and get their bucket
money_data['Is_Converging'] = money_data.index.isin(ordered_is_converging_df['Stock'])
money_data['Is_Converging_Bucket'] = money_data.apply(lambda x: ordered_is_converging_df.loc[ordered_is_converging_df['Stock'] == x.name, 'Category'].iloc[-1] if x['Is_Converging'] else None, axis=1)

# Merge with sorted_data to get TAM in lakhs, Money_Flow_Multiplier, Category, Close for selected stocks
money_data = money_data.merge(money_table[columns_to_display], left_index=True, right_index=True, how='left')

money_data = money_data.merge(green_red_day,left_index=True, right_on='Stock', how='left')

# Reset the index to have 'Stock' as a regular column
money_data.reset_index(inplace=True)
money_data.rename(columns={'index': 'Ticker Symbol'}, inplace=True)  # Rename the index column to 'Stock'

#money_data = money_data.drop(columns=['Money_Flow_Multiplier_x', 'TAM in lakhs', 'Category', 'Close_x'])

bucket_order = ['-100,-40', '-40-0', '>0', 'Not Below %D']

criteria = (
    (money_data['TAM in Lakhs'] > 500) & # Volume > 500 lakhs
    (money_data['Money_Flow_Multiplier'] > 0.00) & #Multiplier above 0
    (money_data['Is_Converging']) 
)


# Set 'Is_Converging_Bucket' as a categorical column with the specified order
money_data['Is_Converging_Bucket'] = pd.Categorical(money_data['Is_Converging_Bucket'], categories=bucket_order, ordered=True)

# Sort the DataFrame by 'Is_Converging_Bucket'
money_data = money_data.sort_values(by=['Is_Converging_Bucket','Money_Flow_Multiplier'],ascending=[True,False])

money_data = money_data[criteria]


# Reorder columns to have 'Stock' as the first column
money_data = money_data[['Stock'] + [col for col in money_data.columns if col != 'Stock' and col != 'Ticker Symbol']]

# Display the selected data
print("Selected Data for Listed Stocks with Convergence Status and Buckets:")
print(money_data.to_markdown(index=False, tablefmt="grid"))

Selected Data for Listed Stocks with Convergence Status and Buckets:
+------------+-----------------+------------------------+----------------+-------------------------+----------+----------------+
| Stock      | Is_Converging   | Is_Converging_Bucket   |   TAM in Lakhs |   Money_Flow_Multiplier |    Close | T Day Action   |
+============+=================+========================+================+=========================+==========+================+
| DAVANGERE  | True            | -100,-40               |       6485.78  |              0.634783   |     9.34 | Green Day      |
+------------+-----------------+------------------------+----------------+-------------------------+----------+----------------+
| FILATFASH  | True            | -100,-40               |       1050.58  |              0.32       |     8.03 | Green Day      |
+------------+-----------------+------------------------+----------------+-------------------------+----------+----------------+
| IRB        | True         

C:\Users\dbcin\AppData\Local\Temp\ipykernel_17552\1864782893.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  money_data = money_data[criteria]


In [65]:
# This is the moneyed is_converging data

money_data_spike = pd.DataFrame(index=convergence_spike_list)
columns_to_display = ['TAM in Lakhs', 'Money_Flow_Multiplier', 'Close']

# Check and annotate which stocks are in ordered_is_converging_df and get their bucket
money_data_spike['Convergence_Spikes'] = money_data_spike.index.isin(ordered_convergence_spike_df['Stock'])
money_data_spike['Convergence_Spikes_Bucket'] = money_data_spike.apply(lambda x: ordered_convergence_spike_df.loc[ordered_convergence_spike_df['Stock'] == x.name, 'Category'].iloc[-1] if x['Convergence_Spikes'] else None, axis=1)


# Merge with sorted_data to get TAM in lakhs, Money_Flow_Multiplier, Category, Close for selected stocks
money_data_spike = money_data_spike.merge(money_table[columns_to_display], left_index=True, right_index=True, how='left')


money_data_spike = money_data_spike.merge(green_red_day,left_index=True, right_on='Stock', how='left')

# Reset the index to have 'Stock' as a regular column
money_data_spike.reset_index(inplace=True)
money_data_spike.rename(columns={'index': 'Ticker Symbol'}, inplace=True)  # Rename the index column to 'Stock'


criteria = (
    (money_data_spike['TAM in Lakhs'] > 500) & # Volume > 500 lakhs
    (money_data['Money_Flow_Multiplier'] > 0.00) & #Multiplier above 0
    (money_data_spike['Convergence_Spikes']) 
)

bucket_order = ['-100,-40', '-40-0', '>0', 'Not Below %D']

# Set 'Is_Converging_Bucket' as a categorical column with the specified order
money_data_spike['Convergence_Spikes_Bucket'] = pd.Categorical(money_data_spike['Convergence_Spikes_Bucket'], categories=bucket_order, ordered=True)

# Sort the DataFrame by 'Is_Converging_Bucket'
money_data_spike = money_data_spike.sort_values(by=['Convergence_Spikes_Bucket','Money_Flow_Multiplier'],ascending=[True,False])

money_data_spike = money_data_spike[criteria]


# Reorder columns to have 'Stock' as the first column
money_data_spike = money_data_spike[['Stock'] + [col for col in money_data_spike.columns if col != 'Stock' and col != 'Ticker Symbol']]

# Display the selected data
print("Selected Data for Listed Stocks with Convergence Spikes:")
print(money_data_spike.to_markdown(index=False, tablefmt="grid"))

Selected Data for Listed Stocks with Convergence Spikes:
+------------+----------------------+-----------------------------+----------------+-------------------------+---------+----------------+
| Stock      | Convergence_Spikes   | Convergence_Spikes_Bucket   |   TAM in Lakhs |   Money_Flow_Multiplier |   Close | T Day Action   |
+============+======================+=============================+================+=========================+=========+================+
| TATACONSUM | True                 | -40-0                       |      50508.5   |               0.876881  | 1146.35 | Green Day      |
+------------+----------------------+-----------------------------+----------------+-------------------------+---------+----------------+
| KECL       | True                 | -40-0                       |       1912.69  |               0.697498  |  206.79 | Green Day      |
+------------+----------------------+-----------------------------+----------------+-------------------------+-----

C:\Users\dbcin\AppData\Local\Temp\ipykernel_17552\3431351188.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  money_data_spike = money_data_spike[criteria]


In [97]:
import pandas as pd

columns_to_display = ['TAM in lakhs', 'Money_Flow_Multiplier', 'Category', 'Close']

data_frames_shortlist = {
    'Sorted Fib Levels': sorted_data[columns_to_display],
    'Is_converging + Fib': selected_data,
    'Converging spike + Fib': selected_data_spike,
    'Money + Is_converging': money_data,
    'Money + Convergence spike': money_data_spike,
    'Fringe mf buys': high_multiplier_stocks_sorted
}

current_time = datetime.now().time()

if current_time < datetime.strptime('12:00:00', '%H:%M:%S').time():
    today_date = datetime.today().strftime('%d-%B-%Y')
else:
    today_date = (datetime.today() + timedelta(days=1)).strftime('%d-%B-%Y')

print(today_date)

folder_path = 'C:\\Users\\dbcin\\OneDrive\\Desktop\\Shortlists for stocks'  # Replace with your specific folder path
os.makedirs(folder_path, exist_ok=True)

# Excel file name with today's date
excel_file = os.path.join(folder_path, f'Shortlist for Stocks on {today_date}.xlsx')

# Create a Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:

    # Write each DataFrame to a separate sheet
    for title, df in data_frames_shortlist.items():
        df.to_excel(writer, sheet_name=title, index=True)

print(f'Excel file saved successfully: {excel_file}')

04-July-2024
Excel file saved successfully: C:\Users\dbcin\OneDrive\Desktop\Shortlists\Shortlist for Stocks on 04-July-2024.xlsx
